In [1]:
import cv2
from yolo_predictions import YOLO_Pred

In [2]:
yolo = YOLO_Pred('Model/weights/best.onnx', 'data.yaml')

### Real Time Object Detection

In [3]:
''' # sadece labellıyor
import cv2

cap = cv2.VideoCapture('video.mp4')

# Crop faktörü - ne kadar zoom yapmak istediğinizi belirler
# 0.5 = %50 zoom (merkezi yarısını al)
# 0.3 = %70 zoom (merkezi %30'unu al)
crop_factor = 0.5

while True:
    ret, frame = cap.read()
    if ret == False:
        print('unable to read video')
        break
    
    # Frame boyutlarını al
    height, width = frame.shape[:2]
    
    # Merkezi crop koordinatlarını hesapla
    crop_width = int(width * crop_factor)
    crop_height = int(height * crop_factor)
    
    # Merkezi başlangıç noktalarını hesapla
    start_x = (width - crop_width) // 2
    start_y = (height - crop_height) // 2
    
    # Crop işlemi
    cropped_frame = frame[start_y:start_y + crop_height, start_x:start_x + crop_width]
    
    # İsteğe bağlı: Crop edilmiş frame'i orijinal boyuta resize et
    # Bu sayede görüntü büyütülmüş gibi görünür
    resized_frame = cv2.resize(cropped_frame, (width, height))
    
    # YOLO prediction - resize edilmiş frame'i kullan
    pred_image = yolo.predictions(resized_frame)
    
    # Sonucu göster
    cv2.imshow('YOLO', pred_image)
    
    # ESC tuşu ile çıkış
    if cv2.waitKey(1) & 0xFF == 27:  # ESC tuşu
        break

cap.release()
cv2.destroyAllWindows()
'''

" # sadece labellıyor\nimport cv2\n\ncap = cv2.VideoCapture('video.mp4')\n\n# Crop faktörü - ne kadar zoom yapmak istediğinizi belirler\n# 0.5 = %50 zoom (merkezi yarısını al)\n# 0.3 = %70 zoom (merkezi %30'unu al)\ncrop_factor = 0.5\n\nwhile True:\n    ret, frame = cap.read()\n    if ret == False:\n        print('unable to read video')\n        break\n    \n    # Frame boyutlarını al\n    height, width = frame.shape[:2]\n    \n    # Merkezi crop koordinatlarını hesapla\n    crop_width = int(width * crop_factor)\n    crop_height = int(height * crop_factor)\n    \n    # Merkezi başlangıç noktalarını hesapla\n    start_x = (width - crop_width) // 2\n    start_y = (height - crop_height) // 2\n    \n    # Crop işlemi\n    cropped_frame = frame[start_y:start_y + crop_height, start_x:start_x + crop_width]\n    \n    # İsteğe bağlı: Crop edilmiş frame'i orijinal boyuta resize et\n    # Bu sayede görüntü büyütülmüş gibi görünür\n    resized_frame = cv2.resize(cropped_frame, (width, height))\n 

In [4]:
# video ile görüntü işleme
'''
import cv2
import numpy as np

def validate_dart_circle(image, x, y, r, min_contrast=30, min_inner_darkness=20):
    """
    Tespit edilen çemberin gerçek bir dart olup olmadığını kontrol eder
    """
    # Görüntü sınırlarını kontrol et
    h, w = image.shape[:2]
    if x-r < 0 or x+r >= w or y-r < 0 or y+r >= h:
        return False
    
    # Çember bölgesini al
    circle_region = image[y-r:y+r, x-r:x+r]
    
    if circle_region.size == 0:
        return False
    
    # Gri tona çevir
    if len(circle_region.shape) == 3:
        gray_region = cv2.cvtColor(circle_region, cv2.COLOR_BGR2GRAY)
    else:
        gray_region = circle_region
    
    # Merkez bölgesini kontrol et (dart merkezinin koyu olması gerekir)
    center_size = max(3, r // 4)
    center_region = gray_region[r-center_size:r+center_size, r-center_size:r+center_size]
    
    if center_region.size == 0:
        return False
    
    center_mean = np.mean(center_region)
    
    # Kenar bölgesini kontrol et
    mask = np.zeros(gray_region.shape, dtype=np.uint8)
    cv2.circle(mask, (r, r), r, 255, -1)
    cv2.circle(mask, (r, r), r//2, 0, -1)
    
    edge_pixels = gray_region[mask == 255]
    if len(edge_pixels) == 0:
        return False
    
    edge_mean = np.mean(edge_pixels)
    
    # Kontrast kontrolü
    contrast = abs(edge_mean - center_mean)
    
    # Dart kriterleri:
    # 1. Merkez koyu olmalı
    # 2. Kenar-merkez arasında yeterli kontrast olmalı
    # 3. Çember yeterince düzenli olmalı
    
    return contrast > min_contrast and center_mean < (edge_mean - min_inner_darkness)

def filter_dart_circles(image, circles, yolo_region=None):
    """
    YOLO bölgesi ve dart kriterleri ile çemberleri filtrele
    """
    if len(circles) == 0:
        return []
    
    validated_circles = []
    
    for circle in circles:
        x, y, r = circle
        
        # YOLO bölgesi kontrolü (varsa)
        if yolo_region is not None:
            yolo_x, yolo_y, yolo_w, yolo_h = yolo_region
            if not (yolo_x <= x <= yolo_x + yolo_w and yolo_y <= y <= yolo_y + yolo_h):
                continue
        
        # Dart kriterleri kontrolü
        if validate_dart_circle(image, x, y, r):
            validated_circles.append(circle)
    
    return validated_circles

def extract_yolo_dart_region(yolo_results, image_shape):
    """
    YOLO sonuçlarından dart bölgesini çıkar
    Bu fonksiyon YOLO implementation'ınıza göre güncellenmeli
    """
    # Bu kısım YOLO sonuçlarınızın formatına göre ayarlanmalı
    # Örnek: dart class'ı için bounding box koordinatları
    
    # Şimdilik placeholder - gerçek YOLO sonuçlarınıza göre güncelleyin
    return None

def detect_dart_circles_advanced(image, yolo_results=None, debug=False):
    """
    Gelişmiş dart tespiti - yanlış pozitif filtreleme ile
    """
    # YOLO'dan dart bölgesini çıkar
    yolo_region = extract_yolo_dart_region(yolo_results, image.shape)
    
    # Preprocessing
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Güçlü gürültü azaltma
    denoised = cv2.medianBlur(gray, 5)
    bilateral = cv2.bilateralFilter(denoised, 9, 75, 75)
    
    # Kontrast artırma
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(bilateral)
    
    # Gaussian blur
    blurred = cv2.GaussianBlur(enhanced, (5, 5), 0)
    
    if debug:
        cv2.imshow('Processed for Advanced Detection', blurred)
    
    # Dart boyutuna özel parametreler
    parameter_sets = [
        # Gerçek dart hedefi boyutları
        {"param1": 50, "param2": 25, "minRadius": 8, "maxRadius": 25, "minDist": 25},
        {"param1": 100, "param2": 40, "minRadius": 12, "maxRadius": 35, "minDist": 30},
        {"param1": 80, "param2": 30, "minRadius": 10, "maxRadius": 30, "minDist": 25},
    ]
    
    all_circles = []
    
    for params in parameter_sets:
        circles = cv2.HoughCircles(
            blurred,
            cv2.HOUGH_GRADIENT,
            dp=1,
            minDist=params["minDist"],
            param1=params["param1"],
            param2=params["param2"],
            minRadius=params["minRadius"],
            maxRadius=params["maxRadius"]
        )
        
        if circles is not None:
            circles = np.round(circles[0, :]).astype("int")
            all_circles.extend(circles)
    
    # Çakışan çemberleri temizle
    unique_circles = []
    for circle in all_circles:
        x, y, r = circle
        is_duplicate = False
        for existing in unique_circles:
            ex, ey, er = existing
            distance = np.sqrt((x - ex)**2 + (y - ey)**2)
            if distance < max(r, er) * 0.7:  # %70 overlap threshold
                is_duplicate = True
                break
        if not is_duplicate:
            unique_circles.append(circle)
    
    # Dart kriterleri ile filtrele
    validated_circles = filter_dart_circles(image, unique_circles, yolo_region)
    
    return validated_circles, blurred

# Ana kod
cap = cv2.VideoCapture('video.mp4')
crop_factor = 0.5

while True:
    ret, frame = cap.read()
    if ret == False:
        print('unable to read video')
        break
    
    # Frame boyutlarını al
    height, width = frame.shape[:2]
    
    # Merkezi crop koordinatlarını hesapla
    crop_width = int(width * crop_factor)
    crop_height = int(height * crop_factor)
    
    # Merkezi başlangıç noktalarını hesapla
    start_x = (width - crop_width) // 2
    start_y = (height - crop_height) // 2
    
    # Crop işlemi
    cropped_frame = frame[start_y:start_y + crop_height, start_x:start_x + crop_width]
    
    # İsteğe bağlı: Crop edilmiş frame'i orijinal boyuta resize et
    resized_frame = cv2.resize(cropped_frame, (width, height))
    
    # YOLO prediction - resize edilmiş frame'i kullan
    pred_image = yolo.predictions(resized_frame)
    
    # Gelişmiş dart tespiti
    circles, processed = detect_dart_circles_advanced(resized_frame, debug=True)
    
    # Sonuçları çiz
    result_image = pred_image.copy()
    
    if len(circles) > 0:
        print(f"Found {len(circles)} validated dart circles")
        for i, (x, y, r) in enumerate(circles):
            # Dart çemberini çiz - kalın yeşil
            cv2.circle(result_image, (x, y), r, (0, 255, 0), 3)
            
            # Merkez hedefi - kırmızı
            cv2.circle(result_image, (x, y), 3, (0, 0, 255), -1)
            
            # Çapraz işaret
            cv2.line(result_image, (x-15, y), (x+15, y), (0, 0, 255), 2)
            cv2.line(result_image, (x, y-15), (x, y+15), (0, 0, 255), 2)
            
            # Dart bilgisi
            cv2.putText(result_image, f"DART {i+1}", (x+r+5, y-5), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.putText(result_image, f"({x},{y})", (x+r+5, y+15), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
            
            # Dart skorlama bölgesi (isteğe bağlı)
            # İç çember (bullseye)
            cv2.circle(result_image, (x, y), r//4, (255, 0, 0), 1)
            # Orta çember
            cv2.circle(result_image, (x, y), r//2, (255, 255, 0), 1)
    else:
        cv2.putText(result_image, "No valid darts detected", (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    
    # Tespit sayısını göster
    cv2.putText(result_image, f"Darts: {len(circles)}", (10, height-20), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
    # Ana sonucu göster
    cv2.imshow('YOLO + Validated Dart Detection', result_image)
    
    # ESC tuşu ile çıkış
    if cv2.waitKey(1) & 0xFF == 27:  # ESC tuşu
        break

cap.release()
cv2.destroyAllWindows()
'''

'\nimport cv2\nimport numpy as np\n\ndef validate_dart_circle(image, x, y, r, min_contrast=30, min_inner_darkness=20):\n    """\n    Tespit edilen çemberin gerçek bir dart olup olmadığını kontrol eder\n    """\n    # Görüntü sınırlarını kontrol et\n    h, w = image.shape[:2]\n    if x-r < 0 or x+r >= w or y-r < 0 or y+r >= h:\n        return False\n    \n    # Çember bölgesini al\n    circle_region = image[y-r:y+r, x-r:x+r]\n    \n    if circle_region.size == 0:\n        return False\n    \n    # Gri tona çevir\n    if len(circle_region.shape) == 3:\n        gray_region = cv2.cvtColor(circle_region, cv2.COLOR_BGR2GRAY)\n    else:\n        gray_region = circle_region\n    \n    # Merkez bölgesini kontrol et (dart merkezinin koyu olması gerekir)\n    center_size = max(3, r // 4)\n    center_region = gray_region[r-center_size:r+center_size, r-center_size:r+center_size]\n    \n    if center_region.size == 0:\n        return False\n    \n    center_mean = np.mean(center_region)\n    \n    #

In [5]:
# Kamera ile Dart Tespiti - Optimize Edilmiş Versiyon
import cv2
import numpy as np

# --- FONKSİYONLAR (kırpılmadan) ---
def validate_dart_circle(image, x, y, r, min_contrast=30, min_inner_darkness=20):
    h, w = image.shape[:2]
    if x-r < 0 or x+r >= w or y-r < 0 or y+r >= h:
        return False

    circle_region = image[y-r:y+r, x-r:x+r]
    if circle_region.size == 0:
        return False

    if len(circle_region.shape) == 3:
        gray_region = cv2.cvtColor(circle_region, cv2.COLOR_BGR2GRAY)
    else:
        gray_region = circle_region

    center_size = max(3, r // 4)
    center_region = gray_region[r-center_size:r+center_size, r-center_size:r+center_size]
    if center_region.size == 0:
        return False

    center_mean = np.mean(center_region)

    mask = np.zeros(gray_region.shape, dtype=np.uint8)
    cv2.circle(mask, (r, r), r, 255, -1)
    cv2.circle(mask, (r, r), r//2, 0, -1)

    edge_pixels = gray_region[mask == 255]
    if len(edge_pixels) == 0:
        return False

    edge_mean = np.mean(edge_pixels)
    contrast = abs(edge_mean - center_mean)

    return contrast > min_contrast and center_mean < (edge_mean - min_inner_darkness)

def filter_dart_circles(image, circles, yolo_region=None):
    if len(circles) == 0:
        return []

    validated_circles = []

    for circle in circles:
        x, y, r = circle
        if yolo_region is not None:
            yolo_x, yolo_y, yolo_w, yolo_h = yolo_region
            if not (yolo_x <= x <= yolo_x + yolo_w and yolo_y <= y <= yolo_y + yolo_h):
                continue

        if validate_dart_circle(image, x, y, r):
            validated_circles.append(circle)

    return validated_circles

def extract_yolo_dart_region(yolo_results, image_shape):
    # Bu fonksiyon, eğer YOLO'nun çıktısından bounding box kullanmak istersen özelleştirilmeli.
    return None  # Şimdilik devre dışı

def detect_dart_circles_advanced(image, yolo_results=None, debug=False):
    yolo_region = extract_yolo_dart_region(yolo_results, image.shape)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    denoised = cv2.medianBlur(gray, 5)
    bilateral = cv2.bilateralFilter(denoised, 9, 75, 75)

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(bilateral)

    blurred = cv2.GaussianBlur(enhanced, (5, 5), 0)

    if debug:
        cv2.imshow('Processed for Advanced Detection', blurred)

    parameter_sets = [
        {"param1": 50, "param2": 25, "minRadius": 8, "maxRadius": 25, "minDist": 25},
        {"param1": 100, "param2": 40, "minRadius": 12, "maxRadius": 35, "minDist": 30},
        {"param1": 80, "param2": 30, "minRadius": 10, "maxRadius": 30, "minDist": 25},
    ]

    all_circles = []
    for params in parameter_sets:
        circles = cv2.HoughCircles(
            blurred,
            cv2.HOUGH_GRADIENT,
            dp=1,
            minDist=params["minDist"],
            param1=params["param1"],
            param2=params["param2"],
            minRadius=params["minRadius"],
            maxRadius=params["maxRadius"]
        )
        if circles is not None:
            circles = np.round(circles[0, :]).astype("int")
            all_circles.extend(circles)

    unique_circles = []
    for circle in all_circles:
        x, y, r = circle
        is_duplicate = False
        for existing in unique_circles:
            ex, ey, er = existing
            distance = np.sqrt((x - ex)**2 + (y - ey)**2)
            if distance < max(r, er) * 0.7:
                is_duplicate = True
                break
        if not is_duplicate:
            unique_circles.append(circle)

    validated_circles = filter_dart_circles(image, unique_circles, yolo_region)
    return validated_circles, blurred


# --- ANA KAMERA AKIŞ KODU ---

cap = cv2.VideoCapture(0)
crop_factor = 0.5

while True:
    ret, frame = cap.read()
    if not ret:
        print("Kameradan görüntü alınamadı.")
        break

    height, width = frame.shape[:2]
    crop_width = int(width * crop_factor)
    crop_height = int(height * crop_factor)
    start_x = (width - crop_width) // 2
    start_y = (height - crop_height) // 2

    cropped_frame = frame[start_y:start_y + crop_height, start_x:start_x + crop_width]
    resized_frame = cv2.resize(cropped_frame, (width, height))

    # ↓↓↓ Kendi YOLO sınıf/nesne tahmin fonksiyonun burada olmalı
    pred_image = yolo.predictions(resized_frame)  # <- Bu satırda hata varsa, yolo objesini tanımlamamışsındır

    # Dart tespiti
    circles, processed = detect_dart_circles_advanced(resized_frame, debug=True)

    result_image = pred_image.copy()

    if len(circles) > 0:
        print(f"{len(circles)} adet geçerli dart bulundu.")
        for i, (x, y, r) in enumerate(circles):
            cv2.circle(result_image, (x, y), r, (0, 255, 0), 3)
            cv2.circle(result_image, (x, y), 3, (0, 0, 255), -1)
            cv2.line(result_image, (x-15, y), (x+15, y), (0, 0, 255), 2)
            cv2.line(result_image, (x, y-15), (x, y+15), (0, 0, 255), 2)
            cv2.putText(result_image, f"DART {i+1}", (x+r+5, y-5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            cv2.putText(result_image, f"({x},{y})", (x+r+5, y+15),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
            cv2.circle(result_image, (x, y), r//4, (255, 0, 0), 1)
            cv2.circle(result_image, (x, y), r//2, (255, 255, 0), 1)
    else:
        cv2.putText(result_image, "No valid darts detected", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.putText(result_image, f"Darts: {len(circles)}", (10, height-20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    cv2.imshow("YOLO + Validated Dart Detection", result_image)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


  

  